In [1]:
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import growthepieapi_utils as gtp
sys.path.pop()

# Usage
merged_df = gtp.get_growthepie_api_data()
merged_df = merged_df.reset_index()
print(merged_df.sample(5))


Failed to retrieve data for stables. HTTP Status code: 403


,index,date,chain_name,daa,txcount,txcosts_usd,txcosts_eth,tvl_usd,tvl_eth,fees_usd,fees_eth
3460,3460,2023-06-12,Immutable X,5862.0,143573.0,NaN,NaN,8.217897e+07,4.691318e+04,28784.013572,16.431814
1809,1809,2023-08-27,Optimism,88905.0,311962.0,0.058279,0.000035,2.547262e+09,1.547636e+06,33637.141059,20.436867
3855,3855,2021-02-27,Optimism,NaN,NaN,NaN,NaN,7.450329e+07,5.134657e+04,NaN,NaN
990,990,2023-03-30,Polygon zkEVM,1467.0,7089.0,0.242699,0.000135,1.100102e+06,6.126871e+02,4864.729789,2.709345
2432,2432,2023-01-26,Arbitrum One,43170.0,361429.0,0.109161,0.000068,2.684735e+09,1.662707e+06,50280.647029,31.139752


In [2]:
# Post to Dune API
d.write_dune_api_from_pandas(merged_df, 'growthepie_l2_activity',\
                             'L2 Usage Activity from GrowThePie')

table at: dune_upload.growthepie_l2_activity
Response status code: 200
Response content: b'{"success":true}'
